In [19]:
import cv2
import numpy as np


def ensure_gray(image):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except cv2.error:
        pass
    return image


def ensure_optimal_square(image):
    assert image is not None, image
    nw = nh = cv2.getOptimalDFTSize(max(image.shape[:2]))
    output_image = cv2.copyMakeBorder(
        src=image,
        top=0,
        bottom=nh - image.shape[0],
        left=0,
        right=nw - image.shape[1],
        borderType=cv2.BORDER_CONSTANT,
        value=255,
    )
    return output_image


def get_fft_magnitude(image):
    gray = ensure_gray(image)
    opt_gray = ensure_optimal_square(gray)

    
    opt_gray = cv2.adaptiveThreshold(
        ~opt_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, -10
    )

    
    dft = np.fft.fft2(opt_gray)
    shifted_dft = np.fft.fftshift(dft)

    
    magnitude = np.abs(shifted_dft)
    # magnitude = magnitude / np.max(magnitude)

    return magnitude


def _get_angle_adaptive(m, amax=None, num=None, W=None):
    assert m.shape[0] == m.shape[1]
    r = c = m.shape[0] // 2

    if W is None:
        W = m.shape[0] // 10

    if amax is None:
        amax = 15

    if num is None:
        num = 20

    tr = np.linspace(-1 * amax, amax, amax * num * 2) / 180 * np.pi
    profile_arr = tr.copy()

    def f(t):
        _f = np.vectorize(
            lambda x: m[c + int(x * np.cos(t)), c + int(-1 * x * np.sin(t))]
        )
        _l = _f(range(0, r))
        val_init = np.sum(_l)
        val_correct = np.sum(_l[W:])
        return val_init, val_correct

    vf = np.vectorize(f)

    li = vf(profile_arr)
    li_init = li[0]
    li_correct = li[1]

    a_init = tr[np.argmax(li_init)] / np.pi * 180
    a_correct = tr[np.argmax(li_correct)] / np.pi * 180

    dist = a_init - a_correct
    if a_init < a_correct:
        dist = a_correct - a_init

    return -1 * a_init, -1 * a_correct, dist


def get_angle(image, amax=None, V=None, W=None, D=None, train_D=False):
    assert isinstance(image, np.ndarray), image

    if amax is None:
        amax = 15
    if V is None:
        V = 1024
    if W is None:
        W = 0
    if D is None:
        D = 0.45

    ratio = V / image.shape[0]
    image = cv2.resize(image, None, fx=ratio, fy=ratio)

    magnitude = get_fft_magnitude(image)
    a_init, a_correct, dist = _get_angle_adaptive(magnitude, amax=amax, W=W)

    if train_D is True:
        return a_init, a_correct, dist

    if dist <= D:
        return a_correct
    return a_init

In [20]:
import matplotlib.pyplot as plt

In [21]:
image = cv2.imread("./images/scan_094.png")

rotation_angle = -get_angle(image)  


rows, cols = image.shape[:2]
center = (cols // 2, rows // 2)
rotation_matrix = cv2.getRotationMatrix2D(center, rotation_angle, 1)
final_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))

'''
mask = cv2.cvtColor(final_image, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

# Invert the mask
mask = cv2.bitwise_not(mask)

# Replace black regions with white
final_image[mask != 0] = [255, 255, 255]
'''


cv2.imshow("Original Image", image)
cv2.imshow("Rotated Image", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()